In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Conv1D
from keras.layers import MaxPooling2D, MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from keras.models import load_model

Using TensorFlow backend.


In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_final_labels = np.zeros((train_labels.size, train_labels.max()+1))
train_final_labels[np.arange(train_labels.size),train_labels] = 1

test_final_labels = np.zeros((test_labels.size, test_labels.max()+1))
test_final_labels[np.arange(test_labels.size),test_labels] = 1

In [4]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:
#reshape (28,28,1)
final_train_images = []
for index in range(0, len(train_images)):
    final_train_images.append(train_images[index].reshape((28,28,1)))
final_train_images = np.array(final_train_images)    

test_final_images = []
for index in range(0, len(test_images)):
    test_final_images.append(test_images[index].reshape((28,28,1)))
test_final_images = np.array(test_final_images)    


X_test, X_val, Y_test, Y_val = train_test_split(test_final_images, test_final_labels, test_size = 0.1, random_state = 2)

In [6]:
model = Sequential()
model.add(Conv2D(3,5,activation='relu', input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(9,5,activation='relu',input_shape = (12,12,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 3)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 9)           684       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 9)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 144)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 144)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
batch_size = 32
epochs = 100

In [9]:
history = model.fit(final_train_images, train_final_labels,validation_data=(X_test, Y_test),
                       epochs=epochs, batch_size=batch_size, verbose=1,
                       callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')])

Train on 60000 samples, validate on 9000 samples
Epoch 1/100
60000/60000 [==============================] - 10s 163us/step - loss: 0.7270 - accuracy: 0.7251 - val_loss: 0.5020 - val_accuracy: 0.8152
Epoch 2/100
60000/60000 [==============================] - 8s 138us/step - loss: 0.5087 - accuracy: 0.8120 - val_loss: 0.4267 - val_accuracy: 0.8438
Epoch 3/100
60000/60000 [==============================] - 8s 136us/step - loss: 0.4620 - accuracy: 0.8298 - val_loss: 0.3980 - val_accuracy: 0.8538
Epoch 4/100
60000/60000 [==============================] - 8s 134us/step - loss: 0.4348 - accuracy: 0.8389 - val_loss: 0.3900 - val_accuracy: 0.8588
Epoch 5/100
60000/60000 [==============================] - 8s 139us/step - loss: 0.4096 - accuracy: 0.8488 - val_loss: 0.3747 - val_accuracy: 0.8607
Epoch 6/100
60000/60000 [==============================] - 8s 135us/step - loss: 0.3962 - accuracy: 0.8530 - val_loss: 0.3643 - val_accuracy: 0.8616
Epoch 7/100
60000/60000 [==============================]

In [10]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.30381203916337757 / Test accuracy: 0.8895555734634399


In [11]:
score = model.evaluate(X_val, Y_val, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.30874385857582093 / Test accuracy: 0.890999972820282


In [12]:
model.save('mnist_fashion_adam_cnn.h5')